In [252]:
import numpy as np

class ANN:
    def __init__(self,input_size,hidden_layers,hidden_neurons,output_size,learning_rate):
        self.weights=[]
        self.bias=[]
        self.hidden_layers=hidden_layers
        self.learning_rate=learning_rate;
        
        for i in range(hidden_layers+1):
            if i==0:
                self.weights.append(np.random.randn(hidden_neurons,input_size))
                self.bias.append(np.full((hidden_neurons,1),1))
            elif i==hidden_layers:
                self.weights.append(np.random.randn(output_size,hidden_neurons))
                self.bias.append(np.full((output_size,1),1))
            else:
                self.weights.append(np.random.randn(hidden_neurons,hidden_neurons))
                self.bias.append(np.full((hidden_neurons,1),1))
                
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def first_order_sigmoid(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))
            
    def forward(self,x):
        activations=[]
        activations.append(x)
        for i in range(self.hidden_layers+1):
            x=np.dot(self.weights[i],activations[i])+self.bias[i]
            activations.append(self.sigmoid(x))
        return activations

    def backward(self,activations,di,m):
        delta=(activations[-1]-di.T) * self.first_order_sigmoid(np.dot(self.weights[-1],activations[-2])+self.bias[-1])
        for i in range(self.hidden_layers,-1,-1):
            if i==self.hidden_layers:
                prev=np.array(self.weights[i])
                self.weights[i]=self.weights[i]-(self.learning_rate/m) * np.dot(delta,activations[i].T)
                self.bias[i]=self.bias[i]-(self.learning_rate/m) * np.sum(delta,axis=1,keepdims=True)
            else:
                delta=np.dot(prev.T, delta) * self.first_order_sigmoid(np.dot(self.weights[i], activations[i])+self.bias[i])
                prev=np.array(self.weights[i])
                self.weights[i]=self.weights[i]-(self.learning_rate/m) * np.dot(delta,activations[i].T)
                self.bias[i]=self.bias[i]-(self.learning_rate/m) * np.sum(delta,axis=1,keepdims=True)
            
    def train(self,x,y,epochs):
        for i in range(epochs):
            activations=self.forward(x)
            m=x.shape[1]
            self.backward(activations,y,m)
            if(i%1000==0):
                print("Error at %d epoch : "%(i),np.sum(activations[-1]-y.T))
            
    def predict(self,x):
        predictions=[]
        for input in x:
            prediction = self.forward(np.array(input))
            predictions.append(prediction[-1])
        return predictions
        

In [253]:
input_size=4
hidden_layers=2
neurons_in_hidden_layer=5
output_size=3
learning_rate=0.1

model=ANN(input_size,hidden_layers,neurons_in_hidden_layer,output_size,learning_rate)

In [1]:
import pandas as pd

df=pd.read_csv('Iris.csv')

x = df.drop(['Id','Species'], axis=1)
y=df['Species']

FileNotFoundError: [Errno 2] No such file or directory: 'Iris.csv'

In [ ]:
df

In [255]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(y)

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,encoded_labels,test_size=0.2,random_state=62)

from tensorflow.keras.utils import to_categorical

labels = np.array(y_train)
y_train = to_categorical(labels)
y_train=np.array(y_train)

In [244]:
x_train=x_train.values
x_train=np.array(x_train)
x_train=x_train.T
x_test=x_test.values

In [245]:
epochs=100000
model.train(x_train,y_train,epochs)

Error at 0 epoch :  77.38880130345308
Error at 1000 epoch :  10.090065866692031
Error at 2000 epoch :  9.27301611955386
Error at 3000 epoch :  7.278048796076046
Error at 4000 epoch :  6.018395540878682
Error at 5000 epoch :  5.231747593925679
Error at 6000 epoch :  4.754893665861987
Error at 7000 epoch :  4.4602732597507675
Error at 8000 epoch :  4.167090813240749
Error at 9000 epoch :  3.818712310737296
Error at 10000 epoch :  3.540744923877782
Error at 11000 epoch :  3.450547438871256
Error at 12000 epoch :  3.542934530553627
Error at 13000 epoch :  3.685115680636752
Error at 14000 epoch :  3.723618442165748
Error at 15000 epoch :  3.593160969326122
Error at 16000 epoch :  3.356656881189017
Error at 17000 epoch :  2.754131088754937
Error at 18000 epoch :  2.3326832616868494
Error at 19000 epoch :  2.0063344339631106
Error at 20000 epoch :  1.7627996791253417
Error at 21000 epoch :  1.5812313782506653
Error at 22000 epoch :  1.4433155344544768
Error at 23000 epoch :  1.336089469303710

In [246]:
hh=model.forward([[6.7],[3.0],[5.2],[2.3]])
print(hh[-1])

[[1.69006391e-04]
 [2.35325579e-02]
 [9.85471641e-01]]


In [247]:
test_sample=[]
for i in x_test:
    test_sample.append([[x] for x in i.tolist()])

In [248]:
y_pred=model.predict(test_sample)
y_pred = np.hstack([np.argmax(arr, axis=0) for arr in y_pred]).flatten()

In [249]:
print(y_test)

[2 2 0 0 2 1 2 0 1 1 2 2 0 1 1 1 0 1 0 0 0 0 2 1 2 0 2 0 2 0]


In [250]:
print(y_pred)

[2 2 0 0 2 2 2 0 1 1 2 2 0 1 1 1 0 1 0 0 0 0 2 1 2 0 2 0 2 0]


In [251]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,confusion_matrix
accuracy=accuracy_score(y_test,y_pred)
precision=precision_score(y_test,y_pred,average='weighted')
recall=recall_score(y_test,y_pred,average='weighted')
f1score=f1_score(y_test,y_pred,average='weighted')
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)

Accuracy: 0.9666666666666667
Precision: 0.9696969696969696
Recall: 0.9666666666666667
F1-score: 0.9663492063492065
